In [1]:
from openpyxl import load_workbook
import pandas as pd
import re
from typing import *
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

COL_NAMES_INDEX_IN_DEBIT_AND_CREDIT = 0
DATA_START_INDEX_IN_DEBIT_AND_CREDIT = 1
DATA_START_INDEX_IN_STMT = 8

def load_xlsx(filename: str = 'data/Xi 2023-24 03_October.xlsx', sheet_name: str = 'Debits') -> pd.DataFrame:

    # Load raw data from the Excel workbook starting from row 6
    book = load_workbook(filename, data_only=False)
    # Load the "Debits" workbook sheet
    sheet = book[sheet_name]

    data = [row for row in sheet.values]
    col_names = list(data[COL_NAMES_INDEX_IN_DEBIT_AND_CREDIT])
    col_names[1] = 'USD'
    data_rows = data[DATA_START_INDEX_IN_DEBIT_AND_CREDIT:]
    df = pd.DataFrame(data_rows, columns=col_names)

    # df = df.drop(columns=['foot'])
    # df = df.drop(df.index[0])

    date_pattern = re.compile(r'^\d{4}-\d{2}-\d{2}.*')
    invalid_dates = df['Date'].apply(lambda x: pd.isnull(x) or not date_pattern.match(str(x)))
    invalid_dates[0] = False
    first_invalid_date_index = invalid_dates.idxmax()
    print(f'First invalid date in Excel row {first_invalid_date_index + DATA_START_INDEX_IN_DEBIT_AND_CREDIT}')
    df = df.iloc[:first_invalid_date_index - 1]
    
    df = df.loc[:, ~df.columns.duplicated()]

    return df

def load_statement(filename: str = 'data/stmt october.xlsx', xlsx_df = load_xlsx()) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Get credit_df and debit_df from the Excel workbook starting from row DATA_START_INDEX
    book = load_workbook(filename, data_only=False)

    # Get the bank statement sheet (converted to xlsx)
    sheet = book.active
    data = [row for row in sheet.values]
    col_names = xlsx_df.columns
    data_rows = data[DATA_START_INDEX_IN_STMT:]
    # Extend each row in data_rows with None until it matches the length of col_names. Don't fill the extended cells yet
    data_rows = [list(row) + [None] * (len(col_names) - len(row)) for row in data_rows]
    df = pd.DataFrame(data_rows, columns=col_names)

    df['USD'] = df['USD'].astype(float)
    credit_df = df[df['USD'] > 0]
    debit_df = df[df['USD'] <= 0]
    return credit_df, debit_df



First invalid date in Excel row 232


In [2]:
excel_debit_df = load_xlsx()
# excel_debit_df[-3:-1].head()
excel_debit_df.head()

First invalid date in Excel row 232


,Date,USD,foot,Category,rcpt,Description,None,data,electricity,gas,vonage,waste,water,food,sum food,acctg,TEP,AILG,IFC,exec,???,licenses,house,hvac,cleaning,rush,social,large,rent
0,None,=SUM(B3:B238),=SUM(H2:AC2)-B2,None,None,None,None,=SUM(H3:H232),=SUM(I3:I232),=SUM(J3:J232),=SUM(K3:K232),=SUM(L3:L232),=SUM(M3:M232),=SUM(N3:N232),=SUM(O3:O232),=SUM(P3:P232),=SUM(Q3:Q232),=SUM(R3:R232),=SUM(S3:S232),=SUM(T3:T232),=SUM(U3:U232),=SUM(V3:V232),=SUM(W3:W232),=SUM(X3:X232),=SUM(Y3:Y232),=SUM(Z3:Z232),=SUM(AA3:AA232),=SUM(AB3:AB232),=SUM(AC3:AC232)
1,2023-07-03 00:00:00,-65.28,=SUM(H3:AC3)-B3,sum food,None,STAR MARKET 0602 06/29 PURCHASE BOSTON MA DEBI...,None,"=IF(H$1=$D3,$B3,0)","=IF(I$1=$D3,$B3,0)","=IF(J$1=$D3,$B3,0)","=IF(K$1=$D3,$B3,0)","=IF(L$1=$D3,$B3,0)","=IF(M$1=$D3,$B3,0)","=IF(N$1=$D3,$B3,0)","=IF(O$1=$D3,$B3,0)","=IF(P$1=$D3,$B3,0)","=IF(Q$1=$D3,$B3,0)","=IF(R$1=$D3,$B3,0)","=IF(S$1=$D3,$B3,0)","=IF(T$1=$D3,$B3,0)","=IF(U$1=$D3,$B3,0)","=IF(V$1=$D3,$B3,0)","=IF(W$1=$D3,$B3,0)","=IF(X$1=$D3,$B3,0)","=IF(Y$1=$D3,$B3,0)","=IF(Z$1=$D3,$B3,0)","=IF(AA$1=$D3,$B3,0)","=IF(AB$1=$D3,$B3,0)","=IF(AC$1=$D3,$B3,0)"
2,2023-07-03 00:00:00,-565.64,=SUM(H4:AC4)-B4,sum food,None,BJS.COM #5490 07/01 PURCHASE 800-257-2582 MA D...,None,"=IF(H$1=$D4,$B4,0)","=IF(I$1=$D4,$B4,0)","=IF(J$1=$D4,$B4,0)","=IF(K$1=$D4,$B4,0)","=IF(L$1=$D4,$B4,0)","=IF(M$1=$D4,$B4,0)","=IF(N$1=$D4,$B4,0)","=IF(O$1=$D4,$B4,0)","=IF(P$1=$D4,$B4,0)","=IF(Q$1=$D4,$B4,0)","=IF(R$1=$D4,$B4,0)","=IF(S$1=$D4,$B4,0)","=IF(T$1=$D4,$B4,0)","=IF(U$1=$D4,$B4,0)","=IF(V$1=$D4,$B4,0)","=IF(W$1=$D4,$B4,0)","=IF(X$1=$D4,$B4,0)","=IF(Y$1=$D4,$B4,0)","=IF(Z$1=$D4,$B4,0)","=IF(AA$1=$D4,$B4,0)","=IF(AB$1=$D4,$B4,0)","=IF(AC$1=$D4,$B4,0)"
3,2023-07-03 00:00:00,-82.69,=SUM(H5:AC5)-B5,sum food,None,GROCERY WEEE! 07/01 PURCHASE HTTPSWWW.SAYW CA ...,None,"=IF(H$1=$D5,$B5,0)","=IF(I$1=$D5,$B5,0)","=IF(J$1=$D5,$B5,0)","=IF(K$1=$D5,$B5,0)","=IF(L$1=$D5,$B5,0)","=IF(M$1=$D5,$B5,0)","=IF(N$1=$D5,$B5,0)","=IF(O$1=$D5,$B5,0)","=IF(P$1=$D5,$B5,0)","=IF(Q$1=$D5,$B5,0)","=IF(R$1=$D5,$B5,0)","=IF(S$1=$D5,$B5,0)","=IF(T$1=$D5,$B5,0)","=IF(U$1=$D5,$B5,0)","=IF(V$1=$D5,$B5,0)","=IF(W$1=$D5,$B5,0)","=IF(X$1=$D5,$B5,0)","=IF(Y$1=$D5,$B5,0)","=IF(Z$1=$D5,$B5,0)","=IF(AA$1=$D5,$B5,0)","=IF(AB$1=$D5,$B5,0)","=IF(AC$1=$D5,$B5,0)"
4,2023-07-03 00:00:00,-276.39,=SUM(H6:AC6)-B6,sum food,None,BJS.COM #5490 07/01 PURCHASE 800-257-2582 MA D...,None,"=IF(H$1=$D6,$B6,0)","=IF(I$1=$D6,$B6,0)","=IF(J$1=$D6,$B6,0)","=IF(K$1=$D6,$B6,0)","=IF(L$1=$D6,$B6,0)","=IF(M$1=$D6,$B6,0)","=IF(N$1=$D6,$B6,0)","=IF(O$1=$D6,$B6,0)","=IF(P$1=$D6,$B6,0)","=IF(Q$1=$D6,$B6,0)","=IF(R$1=$D6,$B6,0)","=IF(S$1=$D6,$B6,0)","=IF(T$1=$D6,$B6,0)","=IF(U$1=$D6,$B6,0)","=IF(V$1=$D6,$B6,0)","=IF(W$1=$D6,$B6,0)","=IF(X$1=$D6,$B6,0)","=IF(Y$1=$D6,$B6,0)","=IF(Z$1=$D6,$B6,0)","=IF(AA$1=$D6,$B6,0)","=IF(AB$1=$D6,$B6,0)","=IF(AC$1=$D6,$B6,0)"


In [3]:
credit_df, debit_df = load_statement()
credit_df.head(1)

,Date,USD,foot,Category,rcpt,Description,None,data,electricity,gas,vonage,waste,water,food,sum food,acctg,TEP,AILG,IFC,exec,???,licenses,house,hvac,cleaning,rush,social,large,rent
0,10/2/23,3500.0,None,None,None,Zelle payment from EDEN SOLOMON for '23 Fall r...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [4]:
debit_df.head(1)


,Date,USD,foot,Category,rcpt,Description,None,data,electricity,gas,vonage,waste,water,food,sum food,acctg,TEP,AILG,IFC,exec,???,licenses,house,hvac,cleaning,rush,social,large,rent
4,10/2/23,-43.05,None,None,None,STAR MARKET 0602 09/28 PURCHASE BOSTON MA DEBI...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [5]:
# combine the dataframes of debit excel sheet and bank statement

# write Excel formulas

formula_cols = list(excel_debit_df.columns)
formula_cols = formula_cols[formula_cols.index(None) + 1:]  # formula columns are all columns to the right of an empty (None) column

old_len = excel_debit_df.last_valid_index() + 1

for col in formula_cols:
    last_formula = excel_debit_df.iloc[old_len - 1][col]
    print('LF ' + last_formula)

    # AI - UNCHECKED
    # Get the last row index from the formula
    last_row_index = int(last_formula.split('$')[2].split(',')[0][1:])
    print(last_row_index)

    # Append debit_df to excel_debit_df before populating with formulas
    excel_debit_df = pd.concat([excel_debit_df, debit_df], ignore_index=True)
    # start_index = len(excel_debit_df) - len(debit_df)

    # Populate excel_debit_df with formulas starting at the former length of excel_debit_df
    j = 0
    for i in range(old_len, len(excel_debit_df)):
        new_row_index = last_row_index + j + 1
        new_formula = last_formula.replace(str(last_row_index), str(new_row_index))
        print(new_formula)
        excel_debit_df.loc[i, col] = new_formula
        j += 1

# # Do the same formula replacement for the column "foot"
# col = "foot"
# last_formula = excel_debit_df.iloc[-1][col]
# print(last_formula)

# # Get the last row index from the formula
# last_row_index = int(last_formula.split(':')[0].split('H')[1])
# print(last_row_index)
# print()

# # Populate excel_debit_df with formulas starting at the former length of excel_debit_df
# j = 0
# for i in range(start_index, len(excel_debit_df)):
#     new_row_index = last_row_index + j + 1
#     new_formula = last_formula.replace(str(last_row_index), str(new_row_index))
#     print(new_formula)
#     excel_debit_df.loc[i, col] = new_formula
#     j += 1



LF =IF(H$1=$D231,$B231,0)
231
=IF(H$1=$D232,$B232,0)
=IF(H$1=$D233,$B233,0)
=IF(H$1=$D234,$B234,0)
=IF(H$1=$D235,$B235,0)
=IF(H$1=$D236,$B236,0)
=IF(H$1=$D237,$B237,0)
=IF(H$1=$D238,$B238,0)
=IF(H$1=$D239,$B239,0)
=IF(H$1=$D240,$B240,0)
=IF(H$1=$D241,$B241,0)
=IF(H$1=$D242,$B242,0)
=IF(H$1=$D243,$B243,0)
=IF(H$1=$D244,$B244,0)
=IF(H$1=$D245,$B245,0)
=IF(H$1=$D246,$B246,0)
=IF(H$1=$D247,$B247,0)
=IF(H$1=$D248,$B248,0)
=IF(H$1=$D249,$B249,0)
=IF(H$1=$D250,$B250,0)
=IF(H$1=$D251,$B251,0)
=IF(H$1=$D252,$B252,0)
=IF(H$1=$D253,$B253,0)
=IF(H$1=$D254,$B254,0)
=IF(H$1=$D255,$B255,0)
=IF(H$1=$D256,$B256,0)
=IF(H$1=$D257,$B257,0)
=IF(H$1=$D258,$B258,0)
=IF(H$1=$D259,$B259,0)
=IF(H$1=$D260,$B260,0)
=IF(H$1=$D261,$B261,0)
=IF(H$1=$D262,$B262,0)
=IF(H$1=$D263,$B263,0)
=IF(H$1=$D264,$B264,0)
=IF(H$1=$D265,$B265,0)
=IF(H$1=$D266,$B266,0)
=IF(H$1=$D267,$B267,0)
=IF(H$1=$D268,$B268,0)
=IF(H$1=$D269,$B269,0)
=IF(H$1=$D270,$B270,0)
=IF(H$1=$D271,$B271,0)
=IF(H$1=$D272,$B272,0)
=IF(H$1=$D273,$B273,0)
=IF(

In [6]:
excel_debit_df

,Date,USD,foot,Category,rcpt,Description,None,data,electricity,gas,vonage,waste,water,food,sum food,acctg,TEP,AILG,IFC,exec,???,licenses,house,hvac,cleaning,rush,social,large,rent
0,None,=SUM(B3:B238),=SUM(H2:AC2)-B2,None,None,None,None,=SUM(H3:H232),=SUM(I3:I232),=SUM(J3:J232),=SUM(K3:K232),=SUM(L3:L232),=SUM(M3:M232),=SUM(N3:N232),=SUM(O3:O232),=SUM(P3:P232),=SUM(Q3:Q232),=SUM(R3:R232),=SUM(S3:S232),=SUM(T3:T232),=SUM(U3:U232),=SUM(V3:V232),=SUM(W3:W232),=SUM(X3:X232),=SUM(Y3:Y232),=SUM(Z3:Z232),=SUM(AA3:AA232),=SUM(AB3:AB232),=SUM(AC3:AC232)
1,2023-07-03 00:00:00,-65.28,=SUM(H3:AC3)-B3,sum food,None,STAR MARKET 0602 06/29 PURCHASE BOSTON MA DEBI...,None,"=IF(H$1=$D3,$B3,0)","=IF(I$1=$D3,$B3,0)","=IF(J$1=$D3,$B3,0)","=IF(K$1=$D3,$B3,0)","=IF(L$1=$D3,$B3,0)","=IF(M$1=$D3,$B3,0)","=IF(N$1=$D3,$B3,0)","=IF(O$1=$D3,$B3,0)","=IF(P$1=$D3,$B3,0)","=IF(Q$1=$D3,$B3,0)","=IF(R$1=$D3,$B3,0)","=IF(S$1=$D3,$B3,0)","=IF(T$1=$D3,$B3,0)","=IF(U$1=$D3,$B3,0)","=IF(V$1=$D3,$B3,0)","=IF(W$1=$D3,$B3,0)","=IF(X$1=$D3,$B3,0)","=IF(Y$1=$D3,$B3,0)","=IF(Z$1=$D3,$B3,0)","=IF(AA$1=$D3,$B3,0)","=IF(AB$1=$D3,$B3,0)","=IF(AC$1=$D3,$B3,0)"
2,2023-07-03 00:00:00,-565.64,=SUM(H4:AC4)-B4,sum food,None,BJS.COM #5490 07/01 PURCHASE 800-257-2582 MA D...,None,"=IF(H$1=$D4,$B4,0)","=IF(I$1=$D4,$B4,0)","=IF(J$1=$D4,$B4,0)","=IF(K$1=$D4,$B4,0)","=IF(L$1=$D4,$B4,0)","=IF(M$1=$D4,$B4,0)","=IF(N$1=$D4,$B4,0)","=IF(O$1=$D4,$B4,0)","=IF(P$1=$D4,$B4,0)","=IF(Q$1=$D4,$B4,0)","=IF(R$1=$D4,$B4,0)","=IF(S$1=$D4,$B4,0)","=IF(T$1=$D4,$B4,0)","=IF(U$1=$D4,$B4,0)","=IF(V$1=$D4,$B4,0)","=IF(W$1=$D4,$B4,0)","=IF(X$1=$D4,$B4,0)","=IF(Y$1=$D4,$B4,0)","=IF(Z$1=$D4,$B4,0)","=IF(AA$1=$D4,$B4,0)","=IF(AB$1=$D4,$B4,0)","=IF(AC$1=$D4,$B4,0)"
3,2023-07-03 00:00:00,-82.69,=SUM(H5:AC5)-B5,sum food,None,GROCERY WEEE! 07/01 PURCHASE HTTPSWWW.SAYW CA ...,None,"=IF(H$1=$D5,$B5,0)","=IF(I$1=$D5,$B5,0)","=IF(J$1=$D5,$B5,0)","=IF(K$1=$D5,$B5,0)","=IF(L$1=$D5,$B5,0)","=IF(M$1=$D5,$B5,0)","=IF(N$1=$D5,$B5,0)","=IF(O$1=$D5,$B5,0)","=IF(P$1=$D5,$B5,0)","=IF(Q$1=$D5,$B5,0)","=IF(R$1=$D5,$B5,0)","=IF(S$1=$D5,$B5,0)","=IF(T$1=$D5,$B5,0)","=IF(U$1=$D5,$B5,0)","=IF(V$1=$D5,$B5,0)","=IF(W$1=$D5,$B5,0)","=IF(X$1=$D5,$B5,0)","=IF(Y$1=$D5,$B5,0)","=IF(Z$1=$D5,$B5,0)","=IF(AA$1=$D5,$B5,0)","=IF(AB$1=$D5,$B5,0)","=IF(AC$1=$D5,$B5,0)"
4,2023-07-03 00:00:00,-276.39,=SUM(H6:AC6)-B6,sum food,None,BJS.COM #5490 07/01 PURCHASE 800-257-2582 MA D...,None,"=IF(H$1=$D6,$B6,0)","=IF(I$1=$D6,$B6,0)","=IF(J$1=$D6,$B6,0)","=IF(K$1=$D6,$B6,0)","=IF(L$1=$D6,$B6,0)","=IF(M$1=$D6,$B6,0)","=IF(N$1=$D6,$B6,0)","=IF(O$1=$D6,$B6,0)","=IF(P$1=$D6,$B6,0)","=IF(Q$1=$D6,$B6,0)","=IF(R$1=$D6,$B6,0)","=IF(S$1=$D6,$B6,0)","=IF(T$1=$D6,$B6,0)","=IF(U$1=$D6,$B6,0)","=IF(V$1=$D6,$B6,0)","=IF(W$1=$D6,$B6,0)","=IF(X$1=$D6,$B6,0)","=IF(Y$1=$D6,$B6,0)","=IF(Z$1=$D6,$B6,0)","=IF(AA$1=$D6,$B6,0)","=IF(AB$1=$D6,$B6,0)","=IF(AC$1=$D6,$B6,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,10/30/23,-60.2,None,None,None,AMZN MKTP US*S23X572Q3 10/29 PURCHASE SEATTLE ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"=IF(AC$1=$D292,$B292,0)"
1656,10/30/23,-16.54,None,None,None,AMZN MKTP US*AK0N77H23 10/29 PURCHASE SEATTLE ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"=IF(AC$1=$D293,$B293,0)"
1657,10/30/23,-41.92,None,None,None,WEEE INC. 10/29 PURCHASE 510-573-4967 CA DEBIT...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"=IF(AC$1=$D294,$B294,0)"
1658,10/31/23,-61.72,None,None,None,AMAZON GROCE*TS9UR27M3 10/29 PURCHASE SEATTLE ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"=IF(AC$1=$D295,$B295,0)"
